In [4]:
# import pre-trained model MobileNetV2
from keras.applications import MobileNetV2
from keras import layers, models
import pandas as pd
import os
import requests
import time
import pandas as pd
from urllib.parse import quote
from selenium import webdriver
import sys

sys.path.append('../playerprediction/ml_logic/')

# import functions
from face_scraping import download_images


In [5]:
!pwd


/Users/Jason/Code/Yassinoko/dynamic-players-insights/dynamic-players-insights/notebooks


In [6]:
def download_images(player_name):
    # Create a directory to save the images
    player_folder_path = f"../raw_data/faces/{player_name}"
    os.makedirs(player_folder_path, exist_ok=True)

    # Define the URL for Getty Images
    encoded_player_name = quote(player_name)
    url = f"https://www.gettyimages.be/photos/{encoded_player_name}?assettype=image&compositions=headshot&family=editorial&numberofpeople=one&sort=best"

    # Configure the Chrome driver (Make sure you have the correct chromedriver installed)
    driver = webdriver.Chrome()

    try:
        # Access the URL
        driver.get(url)
        time.sleep(5)  # Allow time for the page to load

        # Extract image URLs
        image_elements = driver.find_elements("xpath", '//img[contains(@src, "https://media.gettyimages.com")]')

        # Download and save 30 images
        count = 0
        for img in image_elements:
            img_url = img.get_attribute("src")
            img_data = requests.get(img_url).content
            with open(f"{player_folder_path}/{player_name}_{count}.jpg", "wb") as handler:
                handler.write(img_data)
            count += 1
            if count == 30:
                break

        print(f"Downloaded {count} images of {player_name} from Getty Images.")

    except Exception as e:
        print(f"Failed to retrieve images of {player_name} from Getty Images. Error: {e}")

    finally:
        # Close the browser
        driver.quit()


In [9]:
# import key_stats file
key_stats = pd.read_csv("../../raw_data/stats/key_stats.csv")

# create dictionary of team_names keys and player_name values
team_names = ['Liverpool', 'Man. City']
# ['Real Madrid', 'Liverpool', 'Man. United', 'Villareal', 'Benfica', 'Bayern', 'Atlético', 'Chelsea']
team_players_dict = {}
for team_name in team_names:
    team_players_dict[team_name] = list(key_stats[key_stats.club == team_name].player_name.unique())

for team, players in team_players_dict.items():
    for player in players:
        download_images(player + ' ' + team)


Downloaded 30 images of Alisson Becker Liverpool from Getty Images.
Downloaded 30 images of Salah Liverpool from Getty Images.
Downloaded 30 images of Fabinho Liverpool from Getty Images.
Downloaded 30 images of Robertson Liverpool from Getty Images.
Downloaded 5 images of Mané Liverpool from Getty Images.
Downloaded 30 images of Van Dijk Liverpool from Getty Images.
Downloaded 30 images of Alexander-Arnold Liverpool from Getty Images.
Downloaded 2 images of Konaté Liverpool from Getty Images.
Downloaded 30 images of Henderson Liverpool from Getty Images.
Downloaded 30 images of Matip Liverpool from Getty Images.
Failed to retrieve images of Thiago Alcántara Liverpool from Getty Images. Error: Message: disconnected: not connected to DevTools
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: chrome=119.0.6045.159)
Stacktrace:
0   chromedriver                        0x0000000104fd2004 chromedriver + 4169732
1   chromedriver                

KeyboardInterrupt: 

In [2]:
# define the model

def load_model():
    model = MobileNetV2(
        input_shape=None,
        alpha=1.0,
        include_top=False,
        weights="imagenet",
        input_tensor=None,
        pooling=None,
        classes=39,
        classifier_activation="softmax",
        )
    return model


In [3]:
# set the first layers to be non-trainable
def set_nontrainable_layers(model):
    model.trainable = False
    return model


In [4]:
# add trainable layers

def add_last_layers(model):
    '''Take a pre-trained model, set its parameters as non-trainable, and add additional trainable layers on top'''
    base_model = set_nontrainable_layers(model)
    flatten_layer = layers.Flatten()
    dense_layer = layers.Dense(500, activation='relu')
    prediction_layer = layers.Dense(39, activation='softmax')

    model = models.Sequential([
        base_model,
        flatten_layer,
        dense_layer,
        prediction_layer
    ])
    return model


In [5]:
model = load_model()
model = set_nontrainable_layers(model)
model = add_last_layers(model)


ValueError: The last dimension of the inputs to a Dense layer should be defined. Found None. Full input shape received: (None, None)